In [1]:
import datetime
import decimal
import pydantic
import enum
from typing import *
import warnings
import QuantLib as ql
import pandas as pd
import numpy as np
import logging

import finsec as fs

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(name)s - %(message)s',
    # filename='my_application.log',
    # filemode='a'
)

In [2]:
# fs.fixed_income_objs.Period(period='1y').model_dump()

## autoeload info:
%load_ext autoreload
%autoreload 2

In [3]:
# BusinessDayConvention.following.to_ql()
# BusinessDayConvention.from_ql(ql.Following)

In [4]:
import json
import functools
from operator import add, sub, mul

T = TypeVar('T')
ListOrT = Union[List[T], T]

ql.Settings.instance().evaluationDate = ql.Date(3, 1, 2025)

usd_ccy = fs.FiatCurrency( ticker='USD', gsid='USD')
usd = fs.create_reference_from_security(usd_ccy)

eur_ccy = fs.FiatCurrency( ticker='EUR', gsid='EUR')
eur = fs.create_reference_from_security(eur_ccy)

In [5]:
acc1 = fs.AccrualInfo(
    start=datetime.date(2025,1,1),
    # end=datetime.date(2025,11,5),
    end='3m',
    period='1m',
    dc=fs.DayCount.Thirty360,
    front_stub_not_back=False,
)
# acc1.as_ql().dates()
print(len(acc1))
acc1.schedule().to_df()

3


,start,end,frac
0,2025-01-01,2025-02-01,0.083333
1,2025-02-01,2025-03-01,0.083333
2,2025-03-01,2025-04-01,0.083333


In [6]:
print(json.dumps(json.loads(acc1.model_dump_json()), indent=4))
fs.AccrualInfo.model_validate_json(acc1.model_dump_json())

{
    "start": "2025-01-01",
    "end": {
        "period": "3M"
    },
    "dc": "30/360",
    "freq": null,
    "cal_accrual": "null",
    "cal_pay": "null",
    "period": {
        "period": "1M"
    },
    "bdc": "F",
    "front_stub_not_back": false,
    "eom": false
}


AccrualInfo(start=datetime.date(2025, 1, 1), end=Period("3M"), dc=<DayCount.Thirty360: '30/360'>, freq=None, cal_accrual=<Calendar.NULL: 'null'>, cal_pay=<Calendar.NULL: 'null'>, period=Period("1M"), bdc=<BusinessDayConvention.following: 'F'>, front_stub_not_back=False, eom=False)

In [7]:
expr1 = fs.FixedRate(rate=decimal.Decimal('0.05')) * (decimal.Decimal('1')/10) + 10

expr1.get_fixing(None), expr1.is_constant

(Decimal('10.005'), True)

In [8]:
dc = ql.Actual360()

# for x in dir(dc):
#     print(x)

In [9]:
expr1.model_dump()

{'components': [{}, {}], 'operator': <ExprOperator.ADD: 'ADD'>}

In [10]:
ntnl = 1_000_000
fixleg1 = fs.Leg(
    ccy=usd,
    notional=ntnl,
    cpn=fs.FixedRate(rate=decimal.Decimal('0.01')),
    acc=fs.AccrualInfo(
        start=datetime.date(2025,1,1),
        end=datetime.date(2026,1,1),
        dc=fs.DayCount.Thirty360,
        freq=12,
        bdc=fs.BusinessDayConvention.unadjusted,
    )
)
# print(fixleg1)

bnd1 = fs.Bond(
    notional=ntnl,
    leg=fixleg1,
    # settle: datetime.date
    settle_days=1
)
print(bnd1)
ql_bnd1 = bnd1.as_quantlib()

notional=Decimal('1000000') leg=Leg(ccy=SecurityReference(gsid='USD', ticker='USD', security_type=<SecurityType.CURRENCY: 2>, security_subtype=<SecuritySubtype.NATIONAL_FIAT: 505>), notional=Decimal('1000000'), cpn=FixedRate(rate=Decimal('0.01'), is_constant=True, is_float=False), acc=AccrualInfo(start=datetime.date(2025, 1, 1), end=datetime.date(2026, 1, 1), dc=<DayCount.Thirty360: '30/360'>, freq=Decimal('12'), cal_accrual=<Calendar.NULL: 'null'>, cal_pay=<Calendar.NULL: 'null'>, period=None, bdc=<BusinessDayConvention.unadjusted: 'U'>, front_stub_not_back=True, eom=False), pay_delay=None, log=<Logger Leg (DEBUG)>) settle_days=1 credit_index=None face=100 settle=None redemption=None log=<Logger Bond (DEBUG)>


In [11]:
bnd1.cashflows_df().style.format({'amount':'{0:,.2f}'})

2025-09-26 12:48:55,097 - DEBUG - matplotlib - matplotlib data path: /config/workspace/venv/lib/python3.12/site-packages/matplotlib/mpl-data
2025-09-26 12:48:55,104 - DEBUG - matplotlib - CONFIGDIR=/config/.config/matplotlib
2025-09-26 12:48:55,122 - DEBUG - matplotlib - interactive is False
2025-09-26 12:48:55,123 - DEBUG - matplotlib - platform is linux
2025-09-26 12:48:55,246 - DEBUG - matplotlib - CACHEDIR=/config/.cache/matplotlib
2025-09-26 12:48:55,248 - DEBUG - matplotlib.font_manager - Using fontManager instance from /config/.cache/matplotlib/fontlist-v390.json


,start,end,frac,date,amount,has_occurred,rate
0,2025-01-01,2025-02-01,0.083333,2025-02-01,833.33,False,0.010000
1,2025-02-01,2025-03-01,0.083333,2025-03-01,833.33,False,0.010000
2,2025-03-01,2025-04-01,0.083333,2025-04-01,833.33,False,0.010000
3,2025-04-01,2025-05-01,0.083333,2025-05-01,833.33,False,0.010000
4,2025-05-01,2025-06-01,0.083333,2025-06-01,833.33,False,0.010000
5,2025-06-01,2025-07-01,0.083333,2025-07-01,833.33,False,0.010000
6,2025-07-01,2025-08-01,0.083333,2025-08-01,833.33,False,0.010000
7,2025-08-01,2025-09-01,0.083333,2025-09-01,833.33,False,0.010000
8,2025-09-01,2025-10-01,0.083333,2025-10-01,833.33,False,0.010000
9,2025-10-01,2025-11-01,0.083333,2025-11-01,833.33,False,0.010000


In [12]:
bnd1.model_dump()

/config/workspace/venv/lib/python3.12/site-packages/pydantic/main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `decimal` - serialized value may not be as expected [input_value=100, input_type=int])
  return self.__pydantic_serializer__.to_python(


{'notional': Decimal('1000000'),
 'leg': {'ccy': {'gsid': 'USD',
   'ticker': 'USD',
   'security_type': <SecurityType.CURRENCY: 2>,
   'security_subtype': <SecuritySubtype.NATIONAL_FIAT: 505>},
  'notional': Decimal('1000000'),
  'cpn': {'rate': Decimal('0.01'), 'is_constant': True, 'is_float': False},
  'acc': {'start': datetime.date(2025, 1, 1),
   'end': datetime.date(2026, 1, 1),
   'dc': <DayCount.Thirty360: '30/360'>,
   'freq': Decimal('12'),
   'cal_accrual': <Calendar.NULL: 'null'>,
   'cal_pay': <Calendar.NULL: 'null'>,
   'period': None,
   'bdc': <BusinessDayConvention.unadjusted: 'U'>,
   'front_stub_not_back': True,
   'eom': False},
  'pay_delay': None},
 'settle_days': 1,
 'credit_index': None,
 'face': 100,
 'settle': None,
 'redemption': None}

In [13]:
bnd2 = fs.Bond.model_validate_json(bnd1.model_dump_json())
bnd2.model_dump()

/config/workspace/venv/lib/python3.12/site-packages/pydantic/main.py:519: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected `decimal` - serialized value may not be as expected [input_value=100, input_type=int])
  return self.__pydantic_serializer__.to_json(


{'notional': Decimal('1000000'),
 'leg': {'ccy': {'gsid': 'USD',
   'ticker': 'USD',
   'security_type': <SecurityType.CURRENCY: 2>,
   'security_subtype': <SecuritySubtype.NATIONAL_FIAT: 505>},
  'notional': Decimal('1000000'),
  'cpn': {'rate': Decimal('0.01'), 'is_constant': True, 'is_float': False},
  'acc': {'start': datetime.date(2025, 1, 1),
   'end': datetime.date(2026, 1, 1),
   'dc': <DayCount.Thirty360: '30/360'>,
   'freq': Decimal('12'),
   'cal_accrual': <Calendar.NULL: 'null'>,
   'cal_pay': <Calendar.NULL: 'null'>,
   'period': None,
   'bdc': <BusinessDayConvention.unadjusted: 'U'>,
   'front_stub_not_back': True,
   'eom': False},
  'pay_delay': None},
 'settle_days': 1,
 'credit_index': None,
 'face': Decimal('100'),
 'settle': None,
 'redemption': None}

In [14]:
bnd1.model_dump()

{'notional': Decimal('1000000'),
 'leg': {'ccy': {'gsid': 'USD',
   'ticker': 'USD',
   'security_type': <SecurityType.CURRENCY: 2>,
   'security_subtype': <SecuritySubtype.NATIONAL_FIAT: 505>},
  'notional': Decimal('1000000'),
  'cpn': {'rate': Decimal('0.01'), 'is_constant': True, 'is_float': False},
  'acc': {'start': datetime.date(2025, 1, 1),
   'end': datetime.date(2026, 1, 1),
   'dc': <DayCount.Thirty360: '30/360'>,
   'freq': Decimal('12'),
   'cal_accrual': <Calendar.NULL: 'null'>,
   'cal_pay': <Calendar.NULL: 'null'>,
   'period': None,
   'bdc': <BusinessDayConvention.unadjusted: 'U'>,
   'front_stub_not_back': True,
   'eom': False},
  'pay_delay': None},
 'settle_days': 1,
 'credit_index': None,
 'face': 100,
 'settle': None,
 'redemption': None}

In [15]:
from functools import reduce
from operator import add

# help(reduce)
fs.Calendar.US_SOFR.bump(datetime.date(2025, 1, 1), '10y')

[datetime.date(2035, 1, 2)]

In [16]:
swp_ntnl = 1_000_000
swp_start = datetime.date(2025,1,3)
swp_end = '3y'
swp_index = 'SOFR'
swp = fs.Swap.make_ois(
    ccy=usd,
    start=swp_start,
    end=swp_end,
    rate=5.0 / 100,
    dc_fix=fs.DayCount.Actual360,
    dc_float=fs.DayCount.Actual360,
    freq_fix=1,
    freq_float=1,
    index=swp_index,
    cal_pay=fs.Calendar.US_SOFR,
    notional=swp_ntnl,
    pay_delay=fs.Period(period='2d'),
)
display(swp.model_dump())

{'legs': ({'ccy': {'gsid': 'USD',
    'ticker': 'USD',
    'security_type': <SecurityType.CURRENCY: 2>,
    'security_subtype': <SecuritySubtype.NATIONAL_FIAT: 505>},
   'notional': Decimal('1000000'),
   'cpn': {'rate': Decimal('0.05'), 'is_constant': True, 'is_float': False},
   'acc': {'start': datetime.date(2025, 1, 3),
    'end': {'period': '3Y'},
    'dc': <DayCount.Actual360: 'act/360'>,
    'freq': Decimal('1'),
    'cal_accrual': <Calendar.NULL: 'null'>,
    'cal_pay': <Calendar.US_SOFR: 'us/sofr'>,
    'period': None,
    'bdc': <BusinessDayConvention.following: 'F'>,
    'front_stub_not_back': True,
    'eom': False},
   'pay_delay': None},
  {'ccy': {'gsid': 'USD',
    'ticker': 'USD',
    'security_type': <SecurityType.CURRENCY: 2>,
    'security_subtype': <SecuritySubtype.NATIONAL_FIAT: 505>},
   'notional': Decimal('-1000000'),
   'cpn': {'index': 'SOFR',
    'type_': {'type_': 'overnight',
     'pay_delay': {'period': '2D'},
     'compounded_not_averaged': True},
    'i

In [17]:
ql_curve_raw = ql.FlatForward(
    0,
    fs.Calendar.US_SOFR.as_ql(),
    ql.QuoteHandle(ql.SimpleQuote(
        0.05,
    )),
    ql.Actual360(),
)

ql_curve = ql.YieldTermStructureHandle(ql_curve_raw)

swp_engine = ql.DiscountingSwapEngine(ql_curve)
sofr_idx = ql.Sofr(ql_curve)
swp_lookup = {
    'SOFR': sofr_idx,
}


In [18]:
ql_swp = swp.as_quantlib( index_lookup=swp_lookup )

ql_swp.setPricingEngine(swp_engine)
ql_swp.fairRate()

0.05114746391826529

In [19]:
swp.fixed_leg.cashflows_df(index_lookup=swp_lookup)

,start,end,frac,amount,has_occurred,rate
date,,,,,,
2026-01-05,2025-01-03,2026-01-03,1.013889,50694.444444,False,0.05
2027-01-04,2026-01-03,2027-01-03,1.013889,50694.444444,False,0.05
2028-01-03,2027-01-03,2028-01-03,1.013889,50694.444444,False,0.05


In [20]:
swp.cashflows_df( index_lookup=swp_lookup )

,start,end,frac,amount,has_occurred,rate,fix
date,,,,,,,
2026-01-05,2025-01-03,2026-01-03,1.013889,50694.444444,False,0.050000,True
2026-01-05,2025-01-03,2026-01-03,1.013889,-52001.419591,False,0.051289,False
2027-01-04,2026-01-03,2027-01-03,1.013889,50694.444444,False,0.050000,True
2027-01-04,2026-01-03,2027-01-03,1.013889,-51709.247697,False,0.051001,False
2028-01-03,2027-01-03,2028-01-03,1.013889,50694.444444,False,0.050000,True
2028-01-03,2027-01-03,2028-01-03,1.013889,-51855.298137,False,0.051145,False


In [21]:
# help(ql.MakeOIS)
ql_swp2 = ql.MakeOIS(
    ql.Period(swp_end),
    sofr_idx,
    0.05,
    paymentLag=2,
    paymentCalendar=ql.UnitedStates( ql.UnitedStates.SOFR ),
    paymentAdjustmentConvention=ql.Following,
    endOfMonth=False,
)
ql_swp2.setPricingEngine(swp_engine)
ql_swp2.fairRate()

0.05128905136099185

In [22]:
print(f'''
my swap: {ql_swp.fairRate():,.3%}
makeois: {ql_swp2.fairRate():,.3%}
diff: {(ql_swp.fairRate()-ql_swp2.fairRate())*10000:+,.2f}bp
''')



my swap: 5.115%
makeois: 5.129%
diff: -1.42bp



In [23]:
# help(ql.DiscountingSwapEngine)
# help(ql.DatedOISRateHelper)
# swp.fixed_leg.as_quantlib()

display(swp.fixed_leg.acc.model_dump())
display(swp.float_leg.acc.model_dump())
# swp.fixed_leg.acc.schedule().model_dump()

fs.Quote(quote=0).handle

{'start': datetime.date(2025, 1, 3),
 'end': {'period': '3Y'},
 'dc': <DayCount.Actual360: 'act/360'>,
 'freq': Decimal('1'),
 'cal_accrual': <Calendar.NULL: 'null'>,
 'cal_pay': <Calendar.US_SOFR: 'us/sofr'>,
 'period': None,
 'bdc': <BusinessDayConvention.following: 'F'>,
 'front_stub_not_back': True,
 'eom': False}

{'start': datetime.date(2025, 1, 3),
 'end': {'period': '3Y'},
 'dc': <DayCount.Actual360: 'act/360'>,
 'freq': Decimal('1'),
 'cal_accrual': <Calendar.NULL: 'null'>,
 'cal_pay': <Calendar.US_SOFR: 'us/sofr'>,
 'period': None,
 'bdc': <BusinessDayConvention.following: 'F'>,
 'front_stub_not_back': True,
 'eom': False}

<QuantLib.QuantLib.QuoteHandle; proxy of <Swig Object of type 'Handle< Quote > *' at 0x7f8ba7556700> >

In [24]:
swp.as_quantlib_helper(index_lookup=swp_lookup)


2025-09-26 12:48:55,903 - DEBUG - Swap - Building actual ql helper


(Quote(quote=<QuantLib.QuantLib.SimpleQuote; proxy of <Swig Object of type 'ext::shared_ptr< SimpleQuote > *' at 0x7f8ba75559e0> >),
 <QuantLib.QuantLib.DatedOISRateHelper; proxy of <Swig Object of type 'ext::shared_ptr< DatedOISRateHelper > *' at 0x7f8ba7554cf0> >)

In [25]:
help(ql.IborIndex)

Help on class IborIndex in module QuantLib.QuantLib:

class IborIndex(InterestRateIndex)
 |  IborIndex(*args)
 |
 |  Proxy of C++ IborIndex class.
 |
 |  Method resolution order:
 |      IborIndex
 |      InterestRateIndex
 |      Index
 |      Observable
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  __init__(self, *args)
 |      __init__(IborIndex self, std::string const & familyName, Period tenor, Integer settlementDays, Currency currency, Calendar calendar, BusinessDayConvention convention, bool endOfMonth, DayCounter dayCounter, YieldTermStructureHandle h=Handle< YieldTermStructure >()) -> IborIndex
 |
 |  __repr__ = _swig_repr(self) from QuantLib.QuantLib
 |
 |  businessDayConvention(self)
 |      businessDayConvention(IborIndex self) -> BusinessDayConvention
 |
 |  clone(self, arg2)
 |      clone(IborIndex self, YieldTermStructureHandle arg2) -> ext::shared_ptr< IborIndex >
 |
 |  endOfMonth(self)
 |      endOfMonth(IborIndex self) -> bool
 |
 |  forwardingTermStruc

## bootstrap a treasury curve

In [26]:
class SingleCurveConfig(pydantic.BaseModel):
    name: Hashable
    date: datetime.date
    settle_days: int
    dc: fs.DayCount
    cal: fs.Calendar

    # parallel_bump:fs.Quote = pydantic.Field(default_factory=fs.Quote)
    resets: Dict[datetime.date, decimal.Decimal] = pydantic.Field(default_factory=dict)
    helpers: Dict[str, Tuple[fs.Quote, Any]] = pydantic.Field(default_factory=dict)

    ql_curve_raw: ql.YieldTermStructure|None=None
    handle: ql.RelinkableYieldTermStructureHandle=pydantic.Field(default_factory=ql.RelinkableYieldTermStructureHandle)
    index: ql.InterestRateIndex|None=None

    class Config:
        arbitrary_types_allowed = True
    
    def benchmarks(self)->Dict[str, fs.Quote]:
        return { n:q[0] for n,q in self.helpers.items() }

    def add_helper(self, name:str, quote:fs.Quote, helper:Any):
        self.helpers[name] = (quote, helper)
    
    def add_reset(self, dt:datetime.date, rate:decimal.Decimal):
        self.resets[dt] = rate
        if self.index is not None:
            self.index.addFixing( ql.Date.from_date(dt), float(rate))
    
    def build_quantlib_index(self):
        usd = ql.USDCurrency()
        self.index = ql.OvernightIndex(
            self.name,
            self.settle_days,
            usd,
            self.cal.as_ql(),
            self.dc.as_ql,
            self.handle
        )
        self.index.addFixings(
            [ql.Date.from_date(x) for x in self.resets.keys()],
            [float(x) for x in self.resets.values()],
        )

    def build_quantlib_curve(self):
        instruments = [x[1] for x in self.helpers.values()]
        self.ql_curve_raw = ql.PiecewiseFlatForward(
            ql.Date.from_date(self.date),
            instruments,
            self.dc.as_ql,
        )
        self.handle.linkTo(self.ql_curve_raw)
    
    # def plot_forwards()

# class CurveInterp(pydantic.BaseModel):
#     order: List[Literal[0,1,2]]
#     joint_date:

In [27]:
tsy_curve = SingleCurveConfig(
    name='USGOVT',
    date=datetime.date(2025, 1, 1),
    settle_days=1,
    dc=fs.DayCount.ActualActual,
    cal=fs.Calendar.US_GovernmentBond,
)

In [28]:
bond_tenors = ['1y','2y','5y','10y']
bonds = [
    fs.Bond(
        credit_index='USGOVT',
        notional=ntnl,
        leg=fs.Leg(
            ccy=usd,
            notional=100,
            cpn=fs.FixedRate(rate=5.0 / 100),
            acc=fs.AccrualInfo(
                start=datetime.date(2025,1,1),
                end=ten,
                dc=fs.DayCount.ActualActual,
                freq=2,
                bdc=fs.BusinessDayConvention.modified_following,
            )
        ),
        settle_days=1
    )
    for ten in bond_tenors
]

# bond_prices = [100] * len(bond_tenors)
bond_prices = [
    100,
    99,
    98,
    97,
]
bond_helpers = [bnd.as_quantlib_helper() for bnd in bonds]
for px, q in zip(bond_prices, bond_helpers):
    q[0].setValue(px)

In [29]:
for ten, qh in zip(bond_tenors, bond_helpers):
    q,h = qh
    tsy_curve.add_helper( ten, q, h)

In [30]:
tsy_curve.build_quantlib_curve()
tsy_curve.ql_curve_raw.enableExtrapolation()

In [31]:
# tsy_curve.plot(datetime.date(2025,1,1), datetime.date(2035,1,1))

In [32]:
# swp_crv0_ql = ql.PiecewiseFlatForward(
# )

# help(ql.PiecewiseFlatForward)
# PiecewiseFlatForward self
#     Date referenceDate
#     RateHelperVector instruments
#     DayCounter dayCounter
#     QuoteHandleVector jumps=std::vector< Handle< Quote > >()
#     DateVector jumpDates=std::vector< Date >()
#     BackwardFlat i=BackwardFlat()
#     IterativeBootstrap b=_IterativeBootstrap()

for x in dir(ql):
    if 'piecewise' in x.lower():
        print(x)

PiecewiseConstantCorrelation
PiecewiseConstantParameter
PiecewiseConvexMonotoneForward
PiecewiseConvexMonotoneZero
PiecewiseCubicZero
PiecewiseFlatForward
PiecewiseFlatHazardRate
PiecewiseForwardSpreadedTermStructure
PiecewiseKrugerLogDiscount
PiecewiseKrugerZero
PiecewiseLinearForward
PiecewiseLinearForwardSpreadedTermStructure
PiecewiseLinearZero
PiecewiseLogCubicDiscount
PiecewiseLogLinearDiscount
PiecewiseLogMixedLinearCubicDiscount
PiecewiseLogParabolicCubicDiscount
PiecewiseMonotonicLogParabolicCubicDiscount
PiecewiseMonotonicParabolicCubicZero
PiecewiseNaturalCubicZero
PiecewiseNaturalLogCubicDiscount
PiecewiseParabolicCubicZero
PiecewiseSplineCubicDiscount
PiecewiseTimeDependentHestonModel
PiecewiseYoYInflation
PiecewiseZeroInflation
PiecewiseZeroSpreadedTermStructure


In [33]:
import salt.curve as crv

today = datetime.date(2025,1,3)
ql.Settings.instance().evaluationDate = ql.Date.from_date(today)

spot_dt = fs.Calendar.US_SOFR.bump( today, '2d')[0]
print(spot_dt)

index_name = 'SOFR'

sofr_curve = crv.SingleCurveConfig(
    ccy=usd,
    name=index_name,

    date=today,
    # date=datetime.date(2025, 1, 7),
    settle_days=2,
    # date=spot_dt,
    # settle_days=0,

    dc=fs.DayCount.Actual360,
    cal=fs.Calendar.US_SOFR,
)
# print(sofr_curve)
curve = crv.MultiCurve(curves={
    index_name:sofr_curve,
})
print(curve)
print(curve.index_lookup)

bm_rates = [
    ('1y',  3.610),
    ('2y',  3.331),
    ('3y',  3.270),
    ('5y',  3.311),
    ('7y',  3.427),
    ('10y', 3.608),
    ('12y', 3.717),
    ('20y', 3.943),
    ('30y', 3.922),
    ('40y', 3.817),
]
# swp_ntnl = 1_000_000

swps = []
quotes = []
rate_mult = 100
bms = dict()
for ten, r in bm_rates:
    swp = fs.Swap.make_ois(
        ccy=usd,
        start=spot_dt,
        end=fs.Period(period=ten),
        rate=r/100,
        dc_fix=fs.DayCount.Actual360,
        dc_float=fs.DayCount.Actual360,
        freq_fix=1,
        freq_float=1,
        index=index_name,
        cal_pay=fs.Calendar.US_SOFR,
        pay_delay=fs.Period(period='2d'),
    )
    quote = fs.Quote(quote=ql.SimpleQuote(r))
    bms[ten] = crv.Benchmark(
        rate=quote,
        instrument=swp,
        risk_mult=1e-2,
        bootstrap_mult=1e-2,
    )
    # _, helper = swp.as_quantlib_helper( quote=quote*1e-4, index_lookup=curve.index_lookup )
    # swps.append(swp)
    # quotes.append(quote)
    # sofr_curve.add_helper( ten, quote, helper )
    # quote.setValue(r*100)
    # helpers.append(helper)

# logging.getLogger().debug('got here')
# sofr_curve.build_quantlib_curve()
sofr_curve.bms = bms
curve.init_all_curves()

curve = crv.MultiCurve.model_validate_json( curve.model_dump_json() )

2025-09-26 12:48:56,156 - DEBUG - SingleCurveConfig - building quantlib index for SOFR
2025-09-26 12:48:56,157 - DEBUG - SingleCurveConfig - Done building quantlib index
2025-09-26 12:48:56,158 - DEBUG - MultiCurve - Building all curves
2025-09-26 12:48:56,171 - DEBUG - MultiCurve - Building all curves
2025-09-26 12:48:56,171 - DEBUG - SingleCurveConfig - Building quantlib curve
2025-09-26 12:48:56,172 - DEBUG - salt.curve - Benchmark building 
2025-09-26 12:48:56,172 - DEBUG - Swap - Building actual ql helper
2025-09-26 12:48:56,175 - DEBUG - salt.curve - Benchmark building 
2025-09-26 12:48:56,175 - DEBUG - Swap - Building actual ql helper
2025-09-26 12:48:56,179 - DEBUG - salt.curve - Benchmark building 
2025-09-26 12:48:56,179 - DEBUG - Swap - Building actual ql helper
2025-09-26 12:48:56,183 - DEBUG - salt.curve - Benchmark building 
2025-09-26 12:48:56,184 - DEBUG - Swap - Building actual ql helper
2025-09-26 12:48:56,190 - DEBUG - salt.curve - Benchmark building 
2025-09-26 12:4

2025-01-07
curves={'SOFR': SingleCurveConfig(name='SOFR', ccy=SecurityReference(gsid='USD', ticker='USD', security_type=<SecurityType.CURRENCY: 2>, security_subtype=<SecuritySubtype.NATIONAL_FIAT: 505>), date=datetime.date(2025, 1, 3), settle_days=2, dc=<DayCount.Actual360: 'act/360'>, cal=<Calendar.US_SOFR: 'us/sofr'>, extrapolate=True, virgin=True, bms={}, resets={}, handle=<QuantLib.QuantLib.RelinkableYieldTermStructureHandle; proxy of <Swig Object of type 'RelinkableHandle< YieldTermStructure > *' at 0x7f8ba75543c0> >, ql_curve_raw=None, index=<QuantLib.QuantLib.OvernightIndex; proxy of <Swig Object of type 'ext::shared_ptr< OvernightIndex > *' at 0x7f8ba743af40> >, log=<Logger SingleCurveConfig (DEBUG)>)} log=<Logger MultiCurve (DEBUG)>
{'SOFR': <QuantLib.QuantLib.OvernightIndex; proxy of <Swig Object of type 'ext::shared_ptr< OvernightIndex > *' at 0x7f8ba743af40> >}


2025-09-26 12:48:56,589 - DEBUG - SingleCurveConfig - Linking to handle
2025-09-26 12:48:56,597 - DEBUG - SingleCurveConfig - building quantlib index for SOFR
2025-09-26 12:48:56,598 - DEBUG - SingleCurveConfig - Done building quantlib index
2025-09-26 12:48:56,599 - DEBUG - MultiCurve - Building all curves
2025-09-26 12:48:56,600 - DEBUG - SingleCurveConfig - Building quantlib curve
2025-09-26 12:48:56,601 - DEBUG - salt.curve - Benchmark building 
2025-09-26 12:48:56,602 - DEBUG - Swap - Building actual ql helper
2025-09-26 12:48:56,604 - DEBUG - salt.curve - Benchmark building 
2025-09-26 12:48:56,605 - DEBUG - Swap - Building actual ql helper
2025-09-26 12:48:56,608 - DEBUG - salt.curve - Benchmark building 
2025-09-26 12:48:56,608 - DEBUG - Swap - Building actual ql helper
2025-09-26 12:48:56,612 - DEBUG - salt.curve - Benchmark building 
2025-09-26 12:48:56,613 - DEBUG - Swap - Building actual ql helper
2025-09-26 12:48:56,618 - DEBUG - salt.curve - Benchmark building 
2025-09-26

In [34]:
# default: 'Any' = PydanticUndefined,
# 	*,
# 	default_factory: 'Callable[[], Any] | Callable[[dict[str, Any]], Any] | None' = PydanticUndefined,
# 	alias: 'str | None' = PydanticUndefined,
# 	alias_priority: 'int | None' = PydanticUndefined,
# 	validation_alias: 'str | AliasPath | AliasChoices | None' = PydanticUndefined,
# 	serialization_alias: 'str | None' = PydanticUndefined,
# 	title: 'str | None' = PydanticUndefined,
# 	field_title_generator: 'Callable[[str, FieldInfo], str] | None' = PydanticUndefined,
# 	description: 'str | None' = PydanticUndefined,
# 	examples: 'list[Any] | None' = PydanticUndefined,
# 	exclude: 'bool | None' = PydanticUndefined,
# 	discriminator: 'str | types.Discriminator | None' = PydanticUndefined,
# 	deprecated: 'Deprecated | str | bool | None' = PydanticUndefined,
# 	json_schema_extra: 'JsonDict | Callable[[JsonDict], None] | None' = PydanticUndefined,
# 	frozen: 'bool | None' = PydanticUndefined,
# 	validate_default: 'bool | None' = PydanticUndefined,
# 	repr: 'bool' = PydanticUndefined,
# 	init: 'bool | None' = PydanticUndefined,
# 	init_var: 'bool | None' = PydanticUndefined,
# 	kw_only: 'bool | None' = PydanticUndefined,
# 	pattern: 'str | typing.Pattern[str] | None' = PydanticUndefined,
# 	strict: 'bool | None' = PydanticUndefined,
# 	coerce_numbers_to_str: 'bool | None' = PydanticUndefined,
# 	gt: 'annotated_types.SupportsGt | None' = PydanticUndefined,
# 	ge: 'annotated_types.SupportsGe | None' = PydanticUndefined,
# 	lt: 'annotated_types.SupportsLt | None' = PydanticUndefined,
# 	le: 'annotated_types.SupportsLe | None' = PydanticUndefined,
# 	multiple_of: 'float | None' = PydanticUndefined,
# 	allow_inf_nan: 'bool | None' = PydanticUndefined,
# 	max_digits: 'int | None' = PydanticUndefined,
# 	decimal_places: 'int | None' = PydanticUndefined,
# 	min_length: 'int | None' = PydanticUndefined,
# 	max_length: 'int | None' = PydanticUndefined,
# 	union_mode: "Literal['smart', 'left_to_right']" = PydanticUndefined,
# 	fail_fast: 'bool | None' = PydanticUndefined,
# 	**extra: 'Unpack[_EmptyKwargs]') -> 'Any'

In [35]:
def show_curve(crv):
    c = crv.currentLink()
    print(c)
    # for x in dir(c):
    #     if x[0] != '_':
    #         print(x)
    dt0 = c.referenceDate().to_date()
    dt1 = c.maxDate().to_date()
    print(f'{dt0.strftime("%d-%b-%y")} -> {dt1.strftime("%d-%b-%y")}')
    # print(c.dates())

# show_curve(sofr_curve.handle)

In [36]:
# hlpr = sofr_curve.bms['1y'].create_helper(curve)
# # dir(hlpr)
# hlpr.earliestDate(), hlpr.latestDate(), hlpr.pillarDate(), hlpr.quote().value()

In [37]:
swp_test_raw = fs.Swap.make_ois(
    ccy=usd,
    start=spot_dt,
    end=fs.Period(period='10y'),
    rate=3.6/100,
    dc_fix=fs.DayCount.Actual360,
    dc_float=fs.DayCount.Actual360,
    freq_fix=1,
    freq_float=1,
    index=index_name,
    cal_pay=fs.Calendar.US_SOFR,
    notional=1_000_000,
    pay_delay=fs.Period(period='2d'),
)
# swp_test = swp_test_raw.as_quantlib(index_lookup=curve.index_lookup)
# swp_test.setPricingEngine(ql.DiscountingSwapEngine(sofr_curve.handle))
# sofr_curve.benchmark_rates()

In [38]:
# dir(multi_risk.ql_obj)
# help(multi_risk.ql_obj.NPV)

In [39]:
logging.getLogger().setLevel(logging.DEBUG)

multi_risk = curve.risk_factory(
    swp_test_raw.risk_builder(
        # funding_index=curve.get_curve_handle('SOFR'),
        funding_index='SOFR'
    ),
    eps=1e-5
)
print(f'fair rate: {multi_risk.ql_obj.fairRate():,.3%}')
print('multi risk pv:', multi_risk.pv)
df = multi_risk.curve['SOFR'].as_df()
print(df['risk'].sum())
display(df.style.format({
    'risk':'{0:,.1f}'
}))

multi_risk = curve.risk_factory(
    swp_test_raw.risk_builder(
        # funding_index=curve.get_curve_handle('SOFR'),
        funding_index='SOFR'
    ),
    eps=1e-5
)
print(multi_risk.pv)
df = multi_risk.curve['SOFR'].as_df()
print(df['risk'].sum())
df.style.format({
    'risk':'{0:,.1f}'
})

2025-09-26 12:48:57,151 - DEBUG - MultiCurve - Risk factory
2025-09-26 12:48:57,152 - DEBUG - Swap - funding curve df: 0.223641
2025-09-26 12:48:57,162 - DEBUG - SingleCurveConfig - Building full risk ladder
2025-09-26 12:48:57,162 - DEBUG - SingleCurveConfig - PVing
2025-09-26 12:48:57,163 - DEBUG - SingleCurveConfig - Risking for 1y
2025-09-26 12:48:57,333 - DEBUG - SingleCurveConfig - Risking for 2y
2025-09-26 12:48:57,488 - DEBUG - SingleCurveConfig - Risking for 3y
2025-09-26 12:48:57,646 - DEBUG - SingleCurveConfig - Risking for 5y
2025-09-26 12:48:57,806 - DEBUG - SingleCurveConfig - Risking for 7y
2025-09-26 12:48:57,966 - DEBUG - SingleCurveConfig - Risking for 10y
2025-09-26 12:48:58,131 - DEBUG - SingleCurveConfig - Risking for 12y
2025-09-26 12:48:58,287 - DEBUG - SingleCurveConfig - Risking for 20y
2025-09-26 12:48:58,430 - DEBUG - SingleCurveConfig - Risking for 30y
2025-09-26 12:48:58,568 - DEBUG - SingleCurveConfig - Risking for 40y
2025-09-26 12:48:58,832 - DEBUG - Sin

fair rate: 3.604%
multi risk pv: -368.9646350732073
-842.0378760783933


,rate,risk
1y,3.610000,-0.5
2y,3.331000,0.0
3y,3.270000,0.4
5y,3.311000,-1.8
7y,3.427000,1.4
10y,3.608000,-846.8
12y,3.717000,-0.4
20y,3.943000,5.8
30y,3.922000,0.0
40y,3.817000,-0.0


2025-09-26 12:49:00,494 - DEBUG - MultiCurve - Risk factory
2025-09-26 12:49:00,495 - DEBUG - Swap - funding curve df: 0.223648
2025-09-26 12:49:00,504 - DEBUG - SingleCurveConfig - Building full risk ladder
2025-09-26 12:49:00,504 - DEBUG - SingleCurveConfig - PVing
2025-09-26 12:49:00,505 - DEBUG - SingleCurveConfig - Risking for 1y
2025-09-26 12:49:00,664 - DEBUG - SingleCurveConfig - Risking for 2y
2025-09-26 12:49:00,826 - DEBUG - SingleCurveConfig - Risking for 3y
2025-09-26 12:49:00,983 - DEBUG - SingleCurveConfig - Risking for 5y
2025-09-26 12:49:01,143 - DEBUG - SingleCurveConfig - Risking for 7y
2025-09-26 12:49:01,304 - DEBUG - SingleCurveConfig - Risking for 10y
2025-09-26 12:49:01,468 - DEBUG - SingleCurveConfig - Risking for 12y
2025-09-26 12:49:01,624 - DEBUG - SingleCurveConfig - Risking for 20y
2025-09-26 12:49:01,763 - DEBUG - SingleCurveConfig - Risking for 30y
2025-09-26 12:49:01,901 - DEBUG - SingleCurveConfig - Risking for 40y
2025-09-26 12:49:02,165 - DEBUG - Sin

-368.96463506965665
-842.0378729351796


,rate,risk
1y,3.610000,-0.5
2y,3.331000,0.0
3y,3.270000,0.4
5y,3.311000,-1.8
7y,3.427000,1.4
10y,3.608000,-846.8
12y,3.717000,-0.4
20y,3.943000,5.8
30y,3.922000,0.0
40y,3.817000,-0.0


In [40]:
# curve.model_dump()
# 1/0

In [41]:
swp_test_raw = fs.Swap.make_ois(
    ccy=usd,
    start=spot_dt,
    end=fs.Period(period='8y'),
    rate=r/100,
    dc_fix=fs.DayCount.Actual360,
    dc_float=fs.DayCount.Actual360,
    freq_fix=1,
    freq_float=1,
    index=index_name,
    cal_pay=fs.Calendar.US_SOFR,
    notional=20_000_000,
    pay_delay=fs.Period(period='2d'),
)
# swp_test = swp_test_raw.as_quantlib(index_lookup=curve.index_lookup)
# swp_test.setPricingEngine(ql.DiscountingSwapEngine(sofr_curve.handle))
# sofr_curve.benchmark_rates()

multi_risk = curve.risk_factory(
    swp_test_raw.risk_builder(
        # funding_index=curve.get_curve_handle('SOFR'),
        funding_index='SOFR'
    ),
    eps=1e-5
)
print(multi_risk.pv)
df = multi_risk.curve['SOFR'].as_df()
print(df['risk'].sum())
df.style.format({
    'risk':'{0:,.1f}'
})

2025-09-26 12:49:02,220 - DEBUG - MultiCurve - Risk factory
2025-09-26 12:49:02,221 - DEBUG - Swap - funding curve df: 0.223648
2025-09-26 12:49:02,229 - DEBUG - SingleCurveConfig - Building full risk ladder
2025-09-26 12:49:02,230 - DEBUG - SingleCurveConfig - PVing
2025-09-26 12:49:02,231 - DEBUG - SingleCurveConfig - Risking for 1y
2025-09-26 12:49:02,391 - DEBUG - SingleCurveConfig - Risking for 2y
2025-09-26 12:49:02,546 - DEBUG - SingleCurveConfig - Risking for 3y
2025-09-26 12:49:02,703 - DEBUG - SingleCurveConfig - Risking for 5y
2025-09-26 12:49:02,863 - DEBUG - SingleCurveConfig - Risking for 7y
2025-09-26 12:49:03,024 - DEBUG - SingleCurveConfig - Risking for 10y
2025-09-26 12:49:03,188 - DEBUG - SingleCurveConfig - Risking for 12y
2025-09-26 12:49:03,349 - DEBUG - SingleCurveConfig - Risking for 20y
2025-09-26 12:49:03,489 - DEBUG - SingleCurveConfig - Risking for 30y
2025-09-26 12:49:03,627 - DEBUG - SingleCurveConfig - Risking for 40y
2025-09-26 12:49:03,891 - DEBUG - Sin

441409.36015674844
-14160.901448689401


,rate,risk
1y,3.610000,-16.9
2y,3.331000,-9.9
3y,3.270000,-15.7
5y,3.311000,-93.2
7y,3.427000,"-8,142.8"
10y,3.608000,"-5,941.7"
12y,3.717000,19.2
20y,3.943000,40.4
30y,3.922000,-0.1
40y,3.817000,-0.0


In [42]:
import salt.fx
# help(ql.DiscountingSwapEngine)

help(ql.FxSwapRateHelper)

Help on class FxSwapRateHelper in module QuantLib.QuantLib:

class FxSwapRateHelper(RateHelper)
 |  FxSwapRateHelper(*args)
 |
 |  Proxy of C++ FxSwapRateHelper class.
 |
 |  Method resolution order:
 |      FxSwapRateHelper
 |      RateHelper
 |      Observable
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  __init__(self, *args)
 |      __init__(FxSwapRateHelper self, QuoteHandle fwdPoint, QuoteHandle spotFx, Period tenor, Natural fixingDays, Calendar calendar, BusinessDayConvention convention, bool endOfMonth, bool isFxBaseCurrencyCollateralCurrency, YieldTermStructureHandle collateralCurve, Calendar tradingCalendar=Calendar()) -> FxSwapRateHelper
 |
 |  __repr__ = _swig_repr(self) from QuantLib.QuantLib
 |
 |  adjustmentCalendar(self)
 |      adjustmentCalendar(FxSwapRateHelper self) -> Calendar
 |
 |  businessDayConvention(self)
 |      businessDayConvention(FxSwapRateHelper self) -> BusinessDayConvention
 |
 |  calendar(self)
 |      calendar(FxSwapRateHelper self) ->

In [43]:
# print(json.dumps(
#     json.loads(curve.model_dump_json()),
#     indent=4,
# ))
crv.SingleCurveConfig.model_validate_json(
    sofr_curve.model_dump_json()
)

2025-09-26 12:49:03,953 - DEBUG - SingleCurveConfig - building quantlib index for SOFR
2025-09-26 12:49:03,954 - DEBUG - SingleCurveConfig - Done building quantlib index


SingleCurveConfig(name='SOFR', ccy=SecurityReference(gsid='USD', ticker='USD', security_type=<SecurityType.CURRENCY: 2>, security_subtype=<SecuritySubtype.NATIONAL_FIAT: 505>), date=datetime.date(2025, 1, 3), settle_days=2, dc=<DayCount.Actual360: 'act/360'>, cal=<Calendar.US_SOFR: 'us/sofr'>, extrapolate=True, virgin=True, bms={'1y': Benchmark(rate=Quote(quote=<QuantLib.QuantLib.SimpleQuote; proxy of <Swig Object of type 'ext::shared_ptr< SimpleQuote > *' at 0x7f8ba75548d0> >), instrument=Swap(legs=(Leg(ccy=SecurityReference(gsid='USD', ticker='USD', security_type=<SecurityType.CURRENCY: 2>, security_subtype=<SecuritySubtype.NATIONAL_FIAT: 505>), notional=Decimal('10000'), cpn=FixedRate(rate=Decimal('0.0361'), is_constant=True, is_float=False), acc=AccrualInfo(start=datetime.date(2025, 1, 7), end=Period("1Y"), dc=<DayCount.Actual360: 'act/360'>, freq=Decimal('1'), cal_accrual=<Calendar.NULL: 'null'>, cal_pay=<Calendar.US_SOFR: 'us/sofr'>, period=None, bdc=<BusinessDayConvention.follow

In [44]:
curve2 = crv.MultiCurve.model_validate_json(
    curve.model_dump_json()
)

swp_bldr = swp_test_raw.risk_builder(
    # funding_index=curve.get_curve_handle('SOFR'),
    funding_index='SOFR'
)

multi_risk2 = curve2.risk_factory( swp_bldr, eps=1e-5)
print(f'fair rate: {multi_risk2.ql_obj.fairRate():,.3%}')

# multi_risk2 = curve2.risk_factory( swp_bldr, eps=1e-5)
print(f'fair rate: {multi_risk2.ql_obj.fairRate():,.3%}')
print('multi risk pv:', multi_risk2.pv)
df = multi_risk2.curve['SOFR'].as_df()
print(df['risk'].sum())
display(df.style.format({
    'risk':'{0:,.1f}'
}))

2025-09-26 12:49:03,988 - DEBUG - SingleCurveConfig - building quantlib index for SOFR
2025-09-26 12:49:03,989 - DEBUG - SingleCurveConfig - Done building quantlib index
2025-09-26 12:49:03,990 - DEBUG - MultiCurve - Building all curves
2025-09-26 12:49:03,991 - DEBUG - SingleCurveConfig - Building quantlib curve
2025-09-26 12:49:03,992 - DEBUG - salt.curve - Benchmark building 
2025-09-26 12:49:03,993 - DEBUG - Swap - Building actual ql helper
2025-09-26 12:49:03,995 - DEBUG - salt.curve - Benchmark building 
2025-09-26 12:49:03,996 - DEBUG - Swap - Building actual ql helper
2025-09-26 12:49:03,999 - DEBUG - salt.curve - Benchmark building 
2025-09-26 12:49:03,999 - DEBUG - Swap - Building actual ql helper
2025-09-26 12:49:04,003 - DEBUG - salt.curve - Benchmark building 
2025-09-26 12:49:04,004 - DEBUG - Swap - Building actual ql helper
2025-09-26 12:49:04,010 - DEBUG - salt.curve - Benchmark building 
2025-09-26 12:49:04,010 - DEBUG - Swap - Building actual ql helper
2025-09-26 12:4

fair rate: 3.501%
fair rate: 3.501%
multi risk pv: 441039.40762064885
3685741.375505924


,rate,risk
1y,3.610000,"370,309.9"
2y,3.331000,"369,945.2"
3y,3.270000,"369,936.8"
5y,3.311000,"369,859.4"
7y,3.427000,"361,809.7"
10y,3.608000,"364,010.8"
12y,3.717000,"369,971.7"
20y,3.943000,"369,992.9"
30y,3.922000,"369,952.4"
40y,3.817000,"369,952.5"


In [45]:
ntnl = 1_000_000
bnd2 = fs.Bond(
    notional=ntnl,
    leg=fs.Leg(
        ccy=usd,
        notional=ntnl,
        cpn=fs.FixedRate(rate=decimal.Decimal('0.00')),
        acc=fs.AccrualInfo(
            start=datetime.date(2025,1,1),
            # end=datetime.date(2026,1,1),
            end='7y',
            dc=fs.DayCount.Thirty360,
            freq=2,
            bdc=fs.BusinessDayConvention.unadjusted,
        )
    ),
    # settle: datetime.date
    settle_days=1
)

bnd_bldr = bnd2.risk_builder( funding_index='SOFR')
multi_risk3 = curve2.risk_factory( bnd_bldr)

# print(f'fair rate: {multi_risk3.ql_obj.fairRate():,.3%}')
# multi_risk2 = curve2.risk_factory( swp_bldr, eps=1e-5)
# print(f'fair rate: {multi_risk3.ql_obj.fairRate():,.3%}')

print('multi risk pv:', multi_risk3.pv)
df2 = multi_risk3.curve['SOFR'].as_df()
print(df2['risk'].sum())
display(df2.style.format({
    'risk':'{0:,.1f}'
}))
print(multi_risk3.ql_obj.cleanPrice())

2025-09-26 12:49:06,106 - DEBUG - MultiCurve - Risk factory
2025-09-26 12:49:06,106 - DEBUG - Bond - funding curve df: 0.223648
2025-09-26 12:49:06,107 - DEBUG - SingleCurveConfig - Building full risk ladder
2025-09-26 12:49:06,108 - DEBUG - SingleCurveConfig - PVing


2025-09-26 12:49:06,109 - DEBUG - SingleCurveConfig - Risking for 1y
2025-09-26 12:49:06,273 - DEBUG - SingleCurveConfig - Risking for 2y
2025-09-26 12:49:06,425 - DEBUG - SingleCurveConfig - Risking for 3y
2025-09-26 12:49:06,588 - DEBUG - SingleCurveConfig - Risking for 5y
2025-09-26 12:49:06,754 - DEBUG - SingleCurveConfig - Risking for 7y
2025-09-26 12:49:06,909 - DEBUG - SingleCurveConfig - Risking for 10y
2025-09-26 12:49:07,049 - DEBUG - SingleCurveConfig - Risking for 12y
2025-09-26 12:49:07,187 - DEBUG - SingleCurveConfig - Risking for 20y
2025-09-26 12:49:07,323 - DEBUG - SingleCurveConfig - Risking for 30y
2025-09-26 12:49:07,457 - DEBUG - SingleCurveConfig - Risking for 40y
2025-09-26 12:49:07,710 - DEBUG - SingleCurveConfig - last pv 786938.125768
2025-09-26 12:49:07,711 - DEBUG - salt.curve - returning dataframe


multi risk pv: 786938.12576816
-541.1277990788222


,rate,risk
1y,3.610000,1.4
2y,3.331000,5.5
3y,3.270000,13.3
5y,3.311000,23.1
7y,3.427000,-585.8
10y,3.608000,-0.3
12y,3.717000,1.6
20y,3.943000,0.0
30y,3.922000,0.0
40y,3.817000,0.0


78.70157008229106


In [46]:
import cProfile
# cProfile.run('curve2.risk_factory( bnd_bldr, eps=1e-5)', sort='cumtime')

In [47]:
def new_ccy(name:str, as_reference:bool=True):
    ccy = fs.FiatCurrency(ticker=name, gsid=name)
    if as_reference:
        return fs.create_reference_from_security(ccy)
    else:
        return ccy

usd = new_ccy('USD')
eur = new_ccy('EUR')
eurusd = fs.Quote(quote=1.18)

fx = salt.fx.FxBoard(
    quotes = {
        (eur.gsid, usd.gsid): eurusd,
    }
)
assert fx.get_fx_quote(eur, usd).value() == 1.18
fx.get_fx_quote(usd, eur).value(),1/1.18

(0.8474576271186441, 0.8474576271186441)

In [48]:
# for x in dir(ql):
#     if 'flat' in x.lower():
#         print(x)
list(fs.DayCount)

[<DayCount.Actual360: 'act/360'>,
 <DayCount.Actual365Fixed: 'act/365f'>,
 <DayCount.ActualActual: 'act/act'>,
 <DayCount.Thirty360: '30/360'>]

In [49]:
fx_swap = fs.Swap(
    legs=(
        fs.Leg(
            ccy=usd,
            notional=1_000_000,
            cpn=fs.FixedRate(rate=decimal.Decimal('1.00')),
            acc=fs.AccrualInfo(
                start=datetime.date(2025,1,1),
                # end=datetime.date(2026,1,1),
                end='2y',
                dc=fs.DayCount.Thirty360,
                freq=0,
                bdc=fs.BusinessDayConvention.unadjusted,
            )
        ),
        fs.Leg(
            ccy=eur,
            notional=-1_000_000,
            cpn=fs.FixedRate(rate=decimal.Decimal('1.00')),
            acc=fs.AccrualInfo(
                start=datetime.date(2025,1,1),
                # end=datetime.date(2026,1,1),
                end='2y',
                dc=fs.DayCount.Thirty360,
                freq=0,
                bdc=fs.BusinessDayConvention.unadjusted,
            )
        ),
    )
)


In [ ]:
fix_leg1 = fs.Leg(
    ccy=usd,
    notional=1_000_000,
    cpn=fs.FixedRate(rate=decimal.Decimal('1.00')),
    acc=fs.AccrualInfo(
        start=datetime.date(2025,1,1),
        # end=datetime.date(2026,1,1),
        end='2y',
        dc=fs.DayCount.Thirty360,
        freq=0,
        bdc=fs.BusinessDayConvention.unadjusted,
    )
)

qll1 = fix_leg1.as_quantlib()

fl1_bldr = fix_leg1.risk_builder( funding_index='SOFR')
multi_risk3 = curve2.risk_factory( fl1_bldr )

print('multi risk pv:', multi_risk3.pv)
df2 = multi_risk3.curve['SOFR'].as_df()
print(df2['risk'].sum())
display(df2.style.format({
    'risk':'{0:,.1f}'
}))
# print(multi_risk3.ql_obj.cleanPrice())

2025-09-26 12:52:46,463 - DEBUG - MultiCurve - Risk factory
2025-09-26 12:52:46,464 - DEBUG - Leg - funding curve df: 0.223648
2025-09-26 12:52:46,484 - DEBUG - SingleCurveConfig - Building full risk ladder
2025-09-26 12:52:46,485 - DEBUG - SingleCurveConfig - PVing
2025-09-26 12:52:46,486 - DEBUG - SingleCurveConfig - Risking for 1y
2025-09-26 12:52:46,672 - DEBUG - SingleCurveConfig - Risking for 2y
2025-09-26 12:52:46,823 - DEBUG - SingleCurveConfig - Risking for 3y
2025-09-26 12:52:46,985 - DEBUG - SingleCurveConfig - Risking for 5y
2025-09-26 12:52:47,150 - DEBUG - SingleCurveConfig - Risking for 7y
2025-09-26 12:52:47,305 - DEBUG - SingleCurveConfig - Risking for 10y
2025-09-26 12:52:47,446 - DEBUG - SingleCurveConfig - Risking for 12y
2025-09-26 12:52:47,585 - DEBUG - SingleCurveConfig - Risking for 20y
2025-09-26 12:52:47,723 - DEBUG - SingleCurveConfig - Risking for 30y
2025-09-26 12:52:47,856 - DEBUG - SingleCurveConfig - Risking for 40y
2025-09-26 12:52:48,106 - DEBUG - Sing

multi risk pv: -935887.6472021416
182.83299868926406


,rate,risk
1y,3.610000,0.0
2y,3.331000,184.3
3y,3.270000,0.0
5y,3.311000,-1.5
7y,3.427000,0.0
10y,3.608000,0.0
12y,3.717000,0.0
20y,3.943000,0.0
30y,3.922000,0.0
40y,3.817000,0.0


AttributeError: 'Swap' object has no attribute 'cleanPrice'

In [ ]:
help(ql.Swap)

Help on class Swap in module QuantLib.QuantLib:

class Swap(Instrument)
 |  Swap(*args)
 |
 |  Proxy of C++ Swap class.
 |
 |  Method resolution order:
 |      Swap
 |      Instrument
 |      LazyObject
 |      Observable
 |      builtins.object
 |
 |  Methods defined here:
 |
 |  __init__(self, *args)
 |      __init__(Swap self, Leg firstLeg, Leg secondLeg) -> Swap
 |      __init__(Swap self, LegVector legs, BoolVector payer) -> Swap
 |
 |  __repr__ = _swig_repr(self) from QuantLib.QuantLib
 |
 |  endDiscounts(self, j)
 |      endDiscounts(Swap self, Size j) -> DiscountFactor
 |
 |  leg(self, i)
 |      leg(Swap self, Size i) -> Leg
 |
 |  legBPS(self, k)
 |      legBPS(Swap self, Size k) -> Real
 |
 |  legNPV(self, j)
 |      legNPV(Swap self, Size j) -> Real
 |
 |  maturityDate(self)
 |      maturityDate(Swap self) -> Date
 |
 |  npvDateDiscount(self)
 |      npvDateDiscount(Swap self) -> DiscountFactor
 |
 |  numberOfLegs(self)
 |      numberOfLegs(Swap self) -> Size
 |
 |  payer(s